Imports:

In [1]:
import pandas
import fitz

import json, os

Files and directories:

In [2]:

#Set relative paths
ROOT_DIR = os.path.abspath('')
config_path = '\\'.join([ROOT_DIR, 'config.json'])
file_path = '\\'.join([
    ROOT_DIR,
    r'\absolut\absolut-examples\(1) 51070652_156989S.pdf'
    ])

file_path2 = '\\'.join([
    ROOT_DIR,
    r'\absolut\absolut-examples\(1) 51073982_157038S.pdf'
    ])

with open(file_path2) as fp:
    doc = fitz.open(fp)


Constants:

In [3]:
REF = "Booking No."
PKG = "Sum:"
NWT = "Total gross weight to be used in BL"
EQU = "CONT:"
MRN = "MRN no:"
ABS = "Reference No."

Main function and loop:

In [4]:
total_height = 0.0
total_words = []
ref, pkg, nwt, equ, mrn = "", "", "", "", ""

def alter_values(val, const_val):
    if not val:
        val = page.search_for(const_val)
        try:
            val = val[0]
            val += (0, total_height, 0, total_height)
        except:
            pass
    return val

for page in doc:
    word_list = page.get_text("words")
    for w in word_list:
        total_words.append([w[0], w[1] + total_height, w[2], w[3] + total_height, w[4]])

    ref = alter_values(ref, REF)
    pkg = alter_values(pkg, PKG)
    nwt = alter_values(nwt, NWT)
    equ = alter_values(equ, EQU)
    mrn = alter_values(mrn, MRN)

    total_height += page.rect.height

Get rectangle coordinates:

In [5]:
print(
ref, "\n",
pkg, "\n",
nwt, "\n",
equ, "\n",
mrn
)

Rect(302.0, 65.18310546875, 340.11376953125, 73.00341796875) 
 Rect(110.0, 991.0600128173828, 122.33599853515625, 999.3040161132812) 
 Rect(116.0, 1015.0600128173828, 209.71397399902344, 1023.3040161132812) 
 Rect(32.0, 673.7999877929688, 56.0, 686.2899780273438) 
 Rect(302.0, 709.7999877929688, 344.0, 722.2899780273438)


Adding to base rectangles to extract correct word/value:

In [6]:
#Alter rectangles to get values
ref = ref + (0, 9, 40, 13)
pkg = pkg + (204, 0, 230, 0)
nwt = nwt + (300, 0, 261, 0)
equ = equ + (42, 0, 84, 0)
mrn = mrn + (48, 0, 114, 0)

Intersect two rectangles to get word/value, then run for each of the 5 rectangles:

In [7]:
#Then intersect
def get_word_in_rect(rect):
    return [word[4] for word in total_words if fitz.Rect(word[:4]).intersects(rect)][0]

In [10]:
ref = get_word_in_rect(ref)
pkg = get_word_in_rect(pkg)
nwt = get_word_in_rect(nwt)
equ = get_word_in_rect(equ)
mrn = get_word_in_rect(mrn)

Result:

In [15]:
info = {
    ref: {
        'Container number': equ,
        'Net weight': nwt,
        'MRN': mrn,
        'Packages': pkg
        }
        }

In [16]:
info

{'GOTD00655300': {'Container number': 'CXDU1278300',
  'Net weight': '18778,570',
  'MRN': '23SEE1IWRT2S6LIEU0',
  'Packages': '1145'}}